In [1]:
import time, os, json, PyPDF2, re
import pandas as pd

# PDF -> Text

In [2]:
def read_pdf2022(pdf_path, r, name):
    
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        texts = '\t'.join([i.extract_text() for i in reader.pages])
        lastcode = ''; code = ''
        title = ''; lasttitle = ''
        txt_temp = ''
        
        for txt in texts.split('\n'):
            if 'ITEM' in txt:
                #print(txt)
                spl = ':' if ':' in txt else '-'
                for k in txt.split(spl):
                    #print(k, txt_temp)
                    if 'ITEM ' in k:
                        code = k.strip().replace('.', '')
                        sp = code.split('ITEM')
                        if len(sp)>2:
                            addtext = sp[0]
                            code = 'ITEM '+sp[0]
                        if code.strip() == '': print(k)
                        if code not in r: r[code] = dict()

                    else:
                        title = k.strip()
                        #print(title)
                        #if code not in rec: rec[code] = dict()
                        if title not in r[code]: r[code][title] = []
                        
                #print(lastcode, '/', lasttitle)
                #txt_temp = txt_temp.split('2022_')[0]
                try: r[lastcode][lasttitle].append(name+'\t\t\t'+txt_temp)
                except KeyError:
                    print(name)
                    pass
                lastcode = code
                lasttitle = title
                txt_temp = ''
            else:
                txt_temp = txt_temp+'\n'+txt
        r['last'] = txt_temp

In [3]:
pdf_directory = '/Users/bjcliang/Desktop/Nate/Census/Surveys2022'
record = dict()
record[''] = dict()
record[''][''] = []

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        print(pdf_path, end = '\r')
        
        #if pdf_file == 'IN-51821SU.pdf': break
        read_pdf2022(pdf_path, record, pdf_file)
        #if 'ITEM 8' in record: print(pdf_file, record['ITEM 8'])

In [5]:
with open('/Users/bjcliang/Desktop/Nate/Census/Surveys2022/rec.json', 'w') as json_file:
    json.dump(record, json_file)

# Clean Data

In [1]:
import time, os, json, PyPDF2, re, docx
import pandas as pd

In [2]:
def find_item_number(sentence):
    pattern = r'(ITEM \d+)(.*)'
    match = re.search(pattern, sentence)
    if match:
        return (re.sub(match.group(0), '', sentence), match.group(1), match.group(0))

In [40]:
def general_process(text):
    def replace_repeated_num(text):
        pattern = r'(\b\d\.\s)+'
        def replacement(match):
            number = match.group(1).strip()
            return f'\n{number}'
        return re.sub(pattern, replacement, text)
    
    def replace_repeated_string(text):
        pattern = r'(\b[A-Za-z\s]+[A-Za-z]+\.\s)\1+'

        def replacement(match):
            # Extract the first occurrence
            first_occurrence = match.group(1)
            return f'{first_occurrence.strip()}\n'
        return re.sub(pattern, replacement, text)
    
    
    text = re.sub(r'\s+', ' ', text)
    text = replace_repeated_num(text)
    text = replace_repeated_string(text)
    text = re.sub('\n', ' ', text)
    pattern = r'[\w-]*\.pdf Generated \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}'
    text = re.sub(pattern, 'mmmmm', text)
    text = text.split('mmmmm')[0]
    text = text.split('Do Not Submit')[0]
    #pattern = r'\b(?!ZIP|EIN|CFN|IRS|LONG|DISTANCE|LOCAL|TOTAL)([A-Z]{2,}\b(?:\s[A-Z]{2,}\b)*)'
    #text = re.sub(pattern, r'\n\1', text)
    text = text.split('Select ALL that apply.')[0]
    text = text.split('Description Description Value Value Product Code Product Code')[0]
    text = text.split('Select only ONE.')[0]
    
    text = re.sub('INSTRUCTIONS FOR ADMINISTRATIVE EXPENSES', '\nINSTRUCTIONS FOR ADMINISTRATIVE EXPENSES ', text)
    pattern = r'(\b\w+\. )(\1)+'
    text = re.sub(pattern, r'\1', text)
    pattern = r'(\d+\.\s*[A-Z])'
    text = re.sub(pattern, r'\n\1', text)
    
    pattern = r'(\s+\d+)$'
    text = re.sub(pattern, '', text.strip())
    
    a = 'primary kind of business or activity in 2022?'
    if a in text: text = text.split(a)[0]+a
    text = text.split('Description Value Product Code')[0]
    text = text.split('Products or Services not listed?')[0]
    
    return text.strip()

In [4]:
record = json.load(open('/Users/bjcliang/Desktop/Nate/Census/Surveys2022/rec.json'))
record.keys()

dict_keys(['', 'ITEM 1', 'ITEM 2A', 'ITEM 2B', 'ITEM 2C', 'ITEM 2D', 'ITEM 2E', 'separate locations for the purpose of this report Data for of fices that share a common fleet should be included with the controlling main or branch location for that metropolitan areaITEM 2F', 'ITEM 3A', 'ITEM 3B', 'ITEM 4', 'ITEM 5', 'AITEM 7', 'ITEM 20', 'ITEM 22', 'AITEM 25', 'ITEM 26', 'ITEM 28', '1ITEM 29', 'ITEM 29', 'ITEM 31', 'ITEM 32', 'last', 'ITEM 7', 'ITEM 33', 'ITEM 33A', 'ITEM 33B', 'ITEM 2F', 'ITEM 8', 'ITEM 16', 'AITEM 28', 'BITEM 28', 'ITEM 4A', 'ITEM 4B', 'CITEM 28', 'locations should be included with the controlling headquarters or branch locationITEM 2F', 'medical facilities (eg, anesthesiologists, physical therapists) should not consider these facilities as separate of fice locationsITEM 2F', 'separate establishments Data for these locations should be included with the plant in which the work is actually doneITEM 2F', 'Temporary branch or field of fices should not be considered separa

In [41]:
rec_new = dict()
for item in record:
    if item == '': continue
    if item == 'last': continue
    add, superitem, item_n = find_item_number(item)
    
    if superitem not in rec_new: rec_new[superitem] = dict()
    for title in record[item]:
            
        for txt in record[item][title]:
            #print(txt)
            name, t = txt.split('.pdf\t\t\t\n')
            ntitle = item_n + ': ' + title
            if ntitle not in rec_new[superitem]: rec_new[superitem][ntitle] = dict()
            t = general_process(t)
            if t not in rec_new[superitem][ntitle]: rec_new[superitem][ntitle][t] = []
            rec_new[superitem][ntitle][t].append(name)

In [44]:
from docx.oxml.ns import qn
from docx.shared import Pt

def create_word_document_from_dict(data, path):
    doc = docx.Document()
    doc.add_heading('Economic Census Surveys 2022 (Services)', level=1)
    
    for superitem in sorted(data.keys(), key = lambda x: x[-2:]):
        items = data[superitem]
        if len(data[superitem])==1:
            newitem = list(data[superitem].keys())[0]
            doc.add_heading(newitem, level=2)
            details = data[superitem][newitem]
            
            for key in details:
                doc.add_paragraph(key).style='List Bullet'
                surveys = ', '.join(details[key])
                p = doc.add_paragraph()
                p = p.add_run(f'Surveys included: {surveys}')
                p.font.size = Pt(9)
                #p.italic = True
            
        
        else:
            doc.add_heading(superitem, level=2)
            for item_index, (item, details) in enumerate(items.items(), start=1):
                if len(details) == 1:
                    key = list(details)[0]
                    doc.add_heading(item, level=3)
                    doc.add_paragraph(key).style='List Bullet'
                    surveys = ', '.join(details[key])
                    p = doc.add_paragraph()
                    p = p.add_run(f'Surveys included: {surveys}')
                    p.font.size = Pt(9)
                    
                
                else: 
                    doc.add_heading(item, level=3)
                    #doc.add_paragraph(item).style='List Bullet'

                    for key in details:
                        doc.add_paragraph(key).style='List Bullet 2'
                        surveys = ', '.join(details[key])
                        p = doc.add_paragraph()
                        p = p.add_run(f'Surveys included: {surveys}')
                        p.font.size = Pt(9)
                #p.italic = True   

    # Save the document
    doc.save(path)

create_word_document_from_dict(rec_new, 'Economic Census Surveys 2022.docx')